In [ ]:
import pandas as pd#this is just for seeing that code works but this is not the training code
df1=pd.read_csv("C:\\Users\\Ankit Rai\\Downloads\\stock\\stock_data6.csv")
df2=pd.read_csv("C:\\Users\\Ankit Rai\\Downloads\\stock\\hourly_market_trends_with_directions.csv")

In [2]:
df1
df1['Datetime'] = pd.to_datetime(df1['date']).dt.date

# Convert 'Date' in df2 to datetime object and extract date (just in case, though it may already be in the correct format)
df2['Datetime'] = pd.to_datetime(df2['Timestamp']).dt.date
df = pd.merge(df1, df2, on='Datetime', how='inner')
df
df['Sentiment'] = df['Sentiment'].map({'Positive': 1, 'Negative': -1, 'Neutral': 0})
df['Direction'] = df['Direction'].map({'Increased': 1, 'Decreased': -1, 'No Change': 0})
negative_count = df[df["Sentiment"] == -1].shape[0]
negative_count
df = df.dropna()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from textblob import TextBlob
import re

# Heuristic Features
financial_keywords = ['stock', 'price', 'earnings', 'shares', 'market']

# Ensure 'date' column is parsed as datetime
df['date'] = pd.to_datetime(df['date'])

df['text_length'] = df['message'].apply(len)
df['word_count'] = df['message'].apply(lambda x: len(x.split()))
df['keyword_count'] = df['message'].apply(lambda x: sum(1 for word in financial_keywords if word in x.lower()))
df['has_ticker'] = df['message'].apply(lambda x: 1 if re.search(r'\$\w+', x) else 0)
df['polarity_score'] = df['message'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['hour_of_day'] = df['date'].dt.hour
df['day_of_week'] = df['date'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
df['is_pre_market'] = df['hour_of_day'].apply(lambda x: 1 if x < 9 else 0)
df['is_post_market'] = df['hour_of_day'].apply(lambda x: 1 if x > 16 else 0)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=100)
tfidf_features = tfidf.fit_transform(df['message']).toarray()

# Combine TF-IDF with Heuristic Features
heuristic_features = df[['text_length', 'word_count', 'keyword_count', 'has_ticker', 'polarity_score',
                         'hour_of_day', 'day_of_week', 'is_weekend', 'is_pre_market', 'is_post_market']].values
combined_features = np.hstack((tfidf_features, heuristic_features))

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(combined_features, df['Sentiment'], test_size=0.2, random_state=42)

# Hyperparameter Tuning with GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],    # Minimum samples required at each leaf node
    'max_features': ['sqrt', 'log2']  # Number of features to consider at each split
}

rf = RandomForestClassifier(class_weight='balanced',random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='f1_weighted', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best Parameters
print("Best Parameters:", grid_search.best_params_)

# Model Evaluation
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)
print(classification_report(y_test, y_pred))

Fitting 3 folds for each of 216 candidates, totalling 648 fits
